In [2]:
import pandas as pd

In [3]:
dataFile = 'BX-Book-Ratings.csv'


In [4]:
data = pd.read_csv(dataFile, sep=";", header=0, names=["user", "isbn", "rating"], encoding='latin1')

In [5]:
data.head()

,user,isbn,rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [6]:
bookFile = 'BX-Books.csv'
books = pd.read_csv(bookFile, sep=";", header=0, error_bad_lines=False, usecols=[0,1,2], index_col=0, names=["isbn", "title", "author"], encoding='latin1')

In [7]:
books.head()

,title,author
isbn,,
0195153448,Classical Mythology,Mark P. O. Morford
0002005018,Clara Callan,Richard Bruce Wright
0060973129,Decision in Normandy,Carlo D'Este
0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata
0393045218,The Mummies of Urumchi,E. J. W. Barber


In [8]:
def bookMeta(isbn):
    title = books.at[isbn, "title"]
    author = books.at[isbn, "author"]
    return title, author


In [9]:
bookMeta("0671027360")

('Angels &amp; Demons', 'Dan Brown')

In [10]:
def faveBooks(user, N):
    userRatings = data[data["user"] == user]
    sortedRatings = pd.DataFrame.sort_values(userRatings, ['rating'], ascending=[0])[:N]
    sortedRatings["title"] = sortedRatings["isbn"].apply(bookMeta)
    return sortedRatings

In [11]:
data = data[data["isbn"].isin(books.index)]

In [12]:
faveBooks(204622, 5)

,user,isbn,rating,title
844955,204622,0967560500,10,"(Natural Hormonal Enhancement, Rob Faigin)"
844935,204622,0671027360,10,"(Angels &amp; Demons, Dan Brown)"
844926,204622,0385504209,10,"(The Da Vinci Code, Dan Brown)"
844958,204622,097173660X,9,"(Life After School Explained, Cap &amp; Compass)"
844920,204622,0060935464,9,"(To Kill a Mockingbird, Harper Lee)"


In [13]:
data.shape

(1031175, 3)

In [14]:
usersPerISBN = data.isbn.value_counts()

In [15]:
usersPerISBN.head(10)

0971880107    2502
0316666343    1295
0385504209     883
0060928336     732
0312195516     723
044023722X     647
0142001740     615
067976402X     614
0671027360     586
0446672211     585
Name: isbn, dtype: int64

In [16]:
usersPerISBN.shape

(270170,)

In [17]:
ISBNsPerUser = data.user.value_counts()

In [18]:
ISBNsPerUser.shape

(92107,)

In [19]:
data = data[data["isbn"].isin(usersPerISBN[usersPerISBN>10].index)]

In [20]:
data = data[data["user"].isin(ISBNsPerUser[ISBNsPerUser>10].index)]

In [21]:
userItemRatingMatrix = pd.pivot_table(data, values='rating', index=['user'], columns=['isbn'])

In [22]:
userItemRatingMatrix.head()

isbn,0002005018,0002251760,0002259834,0002558122,0006480764,000648302X,0006485200,000649840X,000651202X,0006512062,...,8845906884,8845915611,8878188212,8885989403,9074336329,9074336469,950491036X,9681500830,9681500954,9871138016
user,,,,,,,,,,,,,,,,,,,,,
8,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
242,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
243,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
254,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
user1 = 204622
user2 = 255489

In [27]:
user1Ratings = userItemRatingMatrix.transpose()[user1]
user1Ratings.head()

isbn
0002005018   NaN
0002251760   NaN
0002259834   NaN
0002558122   NaN
0006480764   NaN
Name: 204622, dtype: float64

In [28]:
user2Ratings = userItemRatingMatrix.transpose()[user2]

In [29]:
from scipy.spatial.distance import hamming
hamming(user1Ratings, user2Ratings)

0.9999352792699502

In [30]:
import numpy as np
def distance(user1, user2):
    try:
        user1Ratings = userItemRatingMatrix.transpose()[user1]
        user2Ratings = userItemRatingMatrix.transpose()[user2]
        distance = hamming(user1Ratings, user2Ratings)
    except:
        distance = np.NaN
        
    return distance

In [31]:
distance(204622, 10118)

0.9998705585399004

In [33]:
user = 204622
allUsers = pd.DataFrame(userItemRatingMatrix.index)
allUsers = allUsers[allUsers.user!=user]
allUsers.head()

,user
0,8
1,99
2,242
3,243
4,254


In [34]:
allUsers["distance"] = allUsers["user"].apply(lambda x: distance(user,x))

In [35]:
allUsers.head()

,user,distance
0,8,1.000000
1,99,1.000000
2,242,0.999935
3,243,0.999935
4,254,1.000000


In [37]:
K = 10
KnearestUsers = allUsers.sort_values(["distance"], ascending=True)["user"][:K]

In [38]:
KnearestUsers

3201     82893
3368     87555
2624     68555
1813     48046
5401    140036
7584    198711
565      16795
8866    232131
239       7346
9693    251422
Name: user, dtype: int64

In [41]:
def nearestNeighbors(user, K=10):
    allUsers = pd.DataFrame(userItemRatingMatrix.index)
    allUsers = allUsers[allUsers.user!=user]
    allUsers["distance"] = allUsers["user"].apply(lambda x: distance(user,x))
    KnearestUsers = allUsers.sort_values(["distance"], ascending=True)["user"][:K]
    return KnearestUsers

In [42]:
KnearestUsers = nearestNeighbors(user)

In [43]:
KnearestUsers

3201     82893
3368     87555
2624     68555
1813     48046
5401    140036
7584    198711
565      16795
8866    232131
239       7346
9693    251422
Name: user, dtype: int64

In [44]:
NNRatings = userItemRatingMatrix[userItemRatingMatrix.index.isin(KnearestUsers)]
NNRatings

isbn,0002005018,0002251760,0002259834,0002558122,0006480764,000648302X,0006485200,000649840X,000651202X,0006512062,...,8845906884,8845915611,8878188212,8885989403,9074336329,9074336469,950491036X,9681500830,9681500954,9871138016
user,,,,,,,,,,,,,,,,,,,,,
7346,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16795,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48046,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
68555,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
82893,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
87555,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
140036,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
198711,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
232131,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
avgRating = NNRatings.apply(np.nanmean).dropna()

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\apply.py:251: RuntimeWarning: Mean of empty slice
  labels=labels)


In [46]:
avgRating.head()

isbn
0007154615    1.5
0020125305    0.0
0020125607    0.0
0020198817    0.0
0020198906    8.0
dtype: float64

In [47]:
booksAlreadyRead = userItemRatingMatrix.transpose()[user].dropna().index
booksAlreadyRead

Index(['006016848X', '0060935464', '0140042598', '0140178724', '0142004278',
       '0380732238', '0385504209', '0425109720', '0425152898', '0440136482',
       '0440241162', '0451191145', '0451197127', '0553096060', '0671027360',
       '0671027387', '0671666258', '0688174574', '0743225708', '076790592X',
       '0785264280', '0786868716', '0802131867', '0802132952', '0971880107',
       '1853260045', '1853260126', '1853260207', '185326041X', '1878424114'],
      dtype='object', name='isbn')

In [48]:
avgRating = avgRating[~avgRating.index.isin(booksAlreadyRead)]

In [49]:
N = 3
topNISBNs = avgRating.sort_values(ascending=True).index[:N]

In [52]:
pd.Series(topNISBNs).apply(bookMeta)

0       (Cat Haiku, Deborah Coates)
1    (Cavedweller, Dorothy Allison)
2    (Chang and Eng, Darin Strauss)
Name: isbn, dtype: object

In [53]:
def topN(user, N=3):
    KnearestUsers = nearestNeighbors(user)
    NNRatings = userItemRatingMatrix[userItemRatingMatrix.index.isin(KnearestUsers)]
    avgRatings = NNRatings.apply(np.nanmean).dropna()
    booksAlreadyRead = userItemRatingMatrix.transpose()[user].dropna().index
    avgRating = avgRatings[~avgRatings.index.isin(booksAlreadyRead)]
    topNISBNs = avgRatings.sort_values(ascending=False).index[:N]
    return pd.Series(topNISBNs).apply(bookMeta)
    

In [55]:
faveBooks(204813, 10)


,user,isbn,rating,title
845417,204813,0399149848,10,"(Birthright, Nora Roberts)"
845407,204813,0385504209,10,"(The Da Vinci Code, Dan Brown)"
845382,204813,0373218036,10,"(Truly, Madly Manhattan, Nora Roberts)"
845359,204813,0142001805,10,"(The Eyre Affair: A Novel, Jasper Fforde)"
845431,204813,0446527793,10,"(The Guardian, Nicholas Sparks)"
845416,204813,0399149392,10,"(Chesapeake Blue (Quinn Brothers (Hardcover)),..."
845432,204813,0446531332,9,"(Nights in Rodanthe, Nicholas Sparks)"
845434,204813,0446606243,9,"(The Tenth Justice, Brad Meltzer)"
845451,204813,0671027360,9,"(Angels &amp; Demons, Dan Brown)"
845433,204813,0446532452,9,"(The Wedding, Nicholas Sparks)"


In [56]:
topN(204813, 10)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\apply.py:251: RuntimeWarning: Mean of empty slice
  labels=labels)


0    (Flight Number 116 Is Down (Point), Caroline B...
1    (Someone to Watch Over Me : A Novel, Judith Mc...
2                   (PEOPLE OF THE LIE, M. Scott Peck)
3    (Exit Wounds : A Novel of Suspense (Jance, Ja)...
4    (The Little Prince (Wordsworth Collection), An...
5                    (Cradle and All, James Patterson)
6                       (Melody (Logan), V.C. Andrews)
7    (Long Walk to Freedom : The Autobiography of N...
8                          (The Shining, Stephen King)
9    (Natural California: A Postcard Book, Not Appl...
Name: isbn, dtype: object